In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

# Initialisering

Først må eg lese fila over alle som er aktive lærarar i Canvas, og sjekke om dei er medlem i eit testemne. Dette vert berre gjort ein gang, og resultatet vert lagra i ei fil. 

Ikkje køyr denne snutten fleire ganger!


In [32]:
data = pd.read_csv("users_filtered.csv")
data['brukarnamn'] = data['login_id'].apply(lambda x: x.split('@')[0])
data = data[data['brukarnamn'].str.isalpha()]
data.head()

,user_id,integration_id,login_id,first_name,last_name,short_name,email,status,brukarnamn
182,fs:203:1002339,NaN,jwin@hvl.no,Julia Noelle,Winslow,NaN,julia.winslow@hvl.no,active,jwin
185,fs:203:100479,NaN,mabe@hvl.no,Magnar,Berge,NaN,magnar.berge@hvl.no,active,mabe
186,fs:203:100485,NaN,gobe@hvl.no,Geir Omar,Berland,NaN,geir.omar.berland@hvl.no,active,gobe
187,fs:203:100523,NaN,bvd@hvl.no,Britt Viola,Danielsen,NaN,britt.viola.danielsen@hvl.no,active,bvd
188,fs:203:100607,NaN,rher@hvl.no,Rune,Herheim,NaN,rune.herheim@hvl.no,active,rher


In [118]:
# for person in data['login_id']:
#     print(f"{person} | ", end="")
#     try:
#         params = {'per_page': 100}
#         url = f"https://hvl.instructure.com/api/v1/accounts/1/users?search_term={person}"
#         respons = requests.get(url, headers=headers, params=params)
#         user_id = respons.json()[0]['id']
#         name = respons.json()[0]['name']
#         print(f"{user_id} | {name} | ", end="")
#         emneliste = []
#         params = {'per_page': 100, 'state': ['unpublished']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         params = {'per_page': 100, 'state': ['available']}
#         url = f"https://hvl.instructure.com/api/v1/users/{user_id}/courses"
#         respons = requests.get(url, headers=headers, params=params)
#         for emne in respons.json():
#             emneliste.append(emne['name'])
#         for e in emneliste: 
#             if 'Testemne' in e:
#                 print(f"{e}, ", end=" ")
#         print("")
#     except:
#         print("Ikkje lærar.") 

## Opprette testemne for alle lærarar som ikkje har det.

Eg må gå gjennom fila frå forrige blokk og plukke ut alle som har "Lag nytt emne" i Kontrollfelt:


In [120]:
data = pd.read_excel("250506 oversiktsfil.xlsx")
data.head()

,login_id,user_id,Namn,Testemne,Kontrollfelt
0,jwin@hvl.no,66156,Julia Noelle Winslow,"Julia Noelle Winslow,",1
1,mabe@hvl.no,4576,Magnar Berge,"Magnar Berge, eigenpåmelding,",1
2,gobe@hvl.no,1611,Geir Omar Berland,"Geir Omar Berland, Mari Synnøve Berge,",1
3,bvd@hvl.no,4401,Britt Viola Danielsen,"Britt Viola Danielsen, Testemne - Dag Ove - n...",1
4,rher@hvl.no,4886,Rune Herheim,"Rune Herheim, Beate Lode,",1


In [121]:
lag_testemne = data[data['Kontrollfelt'] == 'Lag nytt emne']
lag_testemne.head()


,login_id,user_id,Namn,Testemne,Kontrollfelt
25,hebo@hvl.no,13429,Heine Bøe,NaN,Lag nytt emne
42,hdhu@hvl.no,75662,Hariom Dhungana,NaN,Lag nytt emne
48,stol@hvl.no,41465,Elin Ekkjestøl,NaN,Lag nytt emne
51,thbo@hvl.no,89697,Therese Bøyum,NaN,Lag nytt emne
53,lflo@hvl.no,31748,Lene Ferstad-Løland,NaN,Lag nytt emne


In [134]:
for n in range(11,len(lag_testemne)):
    i = lag_testemne.iloc[n]
    params = {
        'course[name]': f"Testemne for {i['Namn']}",
        'course[sis_course_id]': f"hvl250506-e{n}-5-akv",
        'course[course_code]': f"Testemne for {i['Namn']}",
        'course[syllabus_body]': "Dette er ditt testemne; her kan du gjere alt du vil prøve på i eit 'vanleg' emne.\nTing du lager her kan du hente inn i andre emne til bruk i undervising.",
        'course[default_view]': 'syllabus', 
    }
    url = f"https://hvl.instructure.com/api/v1/accounts/9044/courses"
    response = requests.post(url, headers=headers, params=params)
    course_id = response.json()['id']
    params = {
        'enrollment[user_id]': i['user_id'],
        'enrollment[type]': 'TeacherEnrollment',
        'enrollment[workflow_state]': 'active',
    }
    url = f"https://hvl.instructure.com/api/v1/courses/{course_id}/enrollments"
    response = requests.post(url, headers=headers, params=params)
    print(f"{i['user_id']}: {response.status_code}")

78896: 200
78071: 200
84029: 200
88052: 200
86048: 200
89733: 200
100186: 200
84827: 200
87405: 200
100551: 200
108280: 200
88375: 200
100815: 200
104663: 200
89014: 200
89359: 200
93201: 200
107499: 200
110531: 200
106714: 200
89088: 200
89089: 200
89087: 200
96359: 200
97129: 200
96563: 200
96871: 200
96999: 200
99709: 200
100137: 200
97555: 200
100612: 200
35226: 200
89719: 200
99252: 200
100115: 200
99747: 200
100317: 200
24389: 200
100259: 200
107501: 200
100772: 200
107900000000000992: 200
100513: 200
101130: 200
100711: 200
100301: 200
107490: 200
100566: 200
107824: 200
107349: 200
100792: 200
108382: 200
109457: 200
109581: 200
108262: 200
111210: 200
5096: 200
107280: 200
21262: 200
49516: 200
1533: 200
65049: 200
110383: 200
100385: 200
107900000000100000: 200
97536: 200
84443: 200
4984: 200
107857: 200
95170: 200
100362: 200
26012: 200
90636: 200
2038: 200
49514: 200
104474: 200
101130: 200
29154: 200
77498: 200
94150: 200
4656: 200
25950: 200
100772: 200
24268: 200
50395: 

In [133]:
len(lag_testemne)

225